In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import imageio as iio
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import pandas as pd

In [2]:
!wget -O Student_Performance.zip https://www.dropbox.com/scl/fi/1lceds9y54dbjoc5eiu0u/Student_Performance.zip?rlkey=ycuxtpgeufmadx9qk4mp9e9z7&dl=0


--2024-12-31 03:23:39--  https://www.dropbox.com/scl/fi/1lceds9y54dbjoc5eiu0u/Student_Performance.zip?rlkey=ycuxtpgeufmadx9qk4mp9e9z7
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5d1ca002ce632323909047ba8d.dl.dropboxusercontent.com/cd/0/inline/ChRmyN6pTfGZTxQdwwD702poVFG81hrbAom3ZnlcfKcYgiZ5iw418cfc-YR9ZCkRQ4G87grjaT_rOyiyaF7PTb5cfASGxxYqJFkTgU6MLKth3sv6sXLmEdwoND9KiUeZIbOrUmYwBTAnitqJDc4h6p_9/file# [following]
--2024-12-31 03:23:40--  https://uc5d1ca002ce632323909047ba8d.dl.dropboxusercontent.com/cd/0/inline/ChRmyN6pTfGZTxQdwwD702poVFG81hrbAom3ZnlcfKcYgiZ5iw418cfc-YR9ZCkRQ4G87grjaT_rOyiyaF7PTb5cfASGxxYqJFkTgU6MLKth3sv6sXLmEdwoND9KiUeZIbOrUmYwBTAnitqJDc4h6p_9/file
Resolving uc5d1ca002ce632323909047ba8d.dl.dropboxusercontent.com (uc5d1ca002ce632323909047ba8d.dl.dropboxusercontent.com)... 162.125.1

In [3]:
!unzip Student_Performance.zip


Archive:  Student_Performance.zip
  inflating: Student_Performance.csv  
  inflating: __MACOSX/._Student_Performance.csv  


In [15]:
df = pd.read_csv("/content/Student_Performance.csv")
df['Extracurricular Activities'] = df['Extracurricular Activities'].astype('category').cat.codes
X = df.drop("Performance Index",axis = 1)
y = df["Performance Index"]

In [16]:
# Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

--

In [17]:
# change it from pandas to numpy to tensor "matrix"
# train--
x_train_tensor = torch.from_numpy(X_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).float()

train_set = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)

In [20]:
#to check
train_set[:][1]
# train_set[:][0] give me x
# train_set[:][1] give me y

tensor([51., 82., 90.,  ..., 43., 67., 64.])

In [21]:
# test
x_test_tensor = torch.from_numpy(X_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).float()
test_set = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

--

In [22]:
# creat the linear model
# nn.sequential = like a pipline
model = nn.Sequential(
    #nn.Linear its a Linear model
    nn.Linear(X.shape[1], 39) #first layer from in to hidden
    ,nn.ReLU()
    ,nn.Linear(39,1) #sec layer hidden to outbut
    # why 1? cuz its real value not class
)


In [30]:
# now make the train loop

# first > loss func
loss_fun = nn.MSELoss()

# optimizer
# w = w - lr* gradiand
optimizer = optim.SGD(model.parameters(),lr = 1e-4)
# ie-4 == 0.0001

epoch = 1 # numper of iterations

for i in range(epoch):
  # zeros the gradiant
  optimizer.zero_grad()
  # get x and y from train set
  x = train_set[:][0]
  y = train_set[:][1]

  # get the prediction
  prediction = model(x)

  # get the loss
  loss = loss_fun(prediction, y)

  # now backward
  loss.backward() # combute the gradiant

  # use optimizer
  optimizer.step() # update the w
  print(f"Epoch{i +1} loss:{loss}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8000])) that is different to the input size (torch.Size([8000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch1 loss:484.8857421875


In [41]:
# use root mean square to assess the model
def rms(y,y_pred):
  return np.sqrt(np.mean(np.square(y-y_pred)))

print(rms(test_set[:][1].detach().numpy(), model(test_set[:][0]).detach().numpy()))
rms(train_set[:][1].detach().numpy(), model(train_set[:][0]).detach().numpy())

21.78664


22.012167

-

-